In [3]:
import torch
import torch.nn as nn

In [33]:
class EncoderRNN(nn.Module):
    
    def __init__(self,hidden_size,embedding,n_layers=1,dropout=0.1):
        """
        Encoder module for seq2seq architechture.
        """
        
        
        super(EncoderRNN,self).__init__()
        
        self.n_layers=n_layers
        self.hidden_size=hidden_size
        self.embedding=embedding
        
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=dropout,bidirectional=True)
        
    def forward(self,input_seq,input_lengths,hidden=None):
        embedded_input=self.embedding(input_seq)
        packed=nn.utils.rnn.pack_padded_sequence(embedded_input,input_lengths)
        outputs,hidden=self.gru(packed,hidden)
        outputs,_=nn.utils.rnn.pad_packed_sequence(outputs)
        
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        
        return outputs,hidden

In [35]:
class DecoderRNN(nn.Module):
    
    def __init__(self,embedding,hidden_size,output_size,n_layers=1,dropout=0.1):
        super().__init__()
        
        self.embedding=embedding
        self.hidden_size=hidden_size
        self.output_size=output_size
        self.n_layers=n_layers
        self.dropout=dropout
        
        self.embedding_dropout=nn.Dropout(dropout)
        self.gru=nn.GRU(hidden_size,hidden_size,n_layers,dropout=dropout)
        
        self.out=nn.Linear(hidden_size,output_size)
        
    def forward(self,input_step,last_hidden,encoder_outputs):
        
        embedded=self.embedding(input_step)
        embedded=self.embedding_dropout(embedded)
        
        rnn_output, hidden=self.gru(embedded,hidden)
        output=self.out(rnn_output)
        output=F.softmax(output,dim=1)
        output=output.view(output.size()[1],-1)
        
        return output,hidden